In [1]:
import folium
from folium.plugins import MiniMap, MeasureControl
from geopy.geocoders import Nominatim
import pandas as pd
import geopandas as gpd

print("Setup Complete")

Setup Complete


In [2]:
shapefile_path = "PH_Adm2_ProvDists/PH_Adm2_ProvDists.shp"
ph_shapefile = gpd.read_file(shapefile_path)
ph_shapefile.head()

,adm1_psgc,adm2_psgc,adm2_en,geo_level,len_crs,area_crs,len_km,area_km2,geometry
0,100000000,102800000,Ilocos Norte,Prov,309785,3276945154,309,3276.0,"POLYGON ((285928.399 2055561.259, 285954.238 2..."
1,100000000,102900000,Ilocos Sur,Prov,452374,2467458323,452,2467.0,"MULTIPOLYGON (((242575.411 1968180.313, 242419..."
2,100000000,103300000,La Union,Prov,262415,1414080983,262,1414.0,"POLYGON ((240302.245 1844607.162, 239281.38 18..."
3,100000000,105500000,Pangasinan,Prov,789136,5161200257,789,5161.0,"MULTIPOLYGON (((171359.2 1820090.799, 171358.1..."
4,200000000,200900000,Batanes,Prov,230060,201280837,230,201.0,"MULTIPOLYGON (((390824.572 2333905.124, 390822..."


In [3]:
ph_shapefile.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   adm1_psgc  88 non-null     int64   
 1   adm2_psgc  88 non-null     int64   
 2   adm2_en    87 non-null     object  
 3   geo_level  86 non-null     object  
 4   len_crs    88 non-null     int64   
 5   area_crs   88 non-null     int64   
 6   len_km     88 non-null     int64   
 7   area_km2   88 non-null     float64 
 8   geometry   88 non-null     geometry
dtypes: float64(1), geometry(1), int64(5), object(2)
memory usage: 6.3+ KB


In [4]:
# 2. Select only ADM2_EN and geometry columns
ph_shapefile = ph_shapefile[["adm2_en", "geo_level", "geometry"]]
# ph_shapefile.drop_duplicates(subset=["ADM2_EN"], inplace=True)
ph_shapefile.head()

,adm2_en,geo_level,geometry
0,Ilocos Norte,Prov,"POLYGON ((285928.399 2055561.259, 285954.238 2..."
1,Ilocos Sur,Prov,"MULTIPOLYGON (((242575.411 1968180.313, 242419..."
2,La Union,Prov,"POLYGON ((240302.245 1844607.162, 239281.38 18..."
3,Pangasinan,Prov,"MULTIPOLYGON (((171359.2 1820090.799, 171358.1..."
4,Batanes,Prov,"MULTIPOLYGON (((390824.572 2333905.124, 390822..."


In [5]:
# # create a csv with the region name, latitude and longitude of the centroid of each region
# ph_shapefile[["adm2_en", "geometry"]].drop_duplicates().apply(
#     lambda row: pd.Series(
#         {
#             "region_name": row["adm2_en"],
#             "latitude": row["geometry"].centroid.y,
#             "longitude": row["geometry"].centroid.x,
#         }
#     ),
#     axis=1,
# ).to_csv("unique_ADM2_EN_data.csv", index=False)

In [6]:
# 3. Create a DataFrame for climate type per region
# Read the CSV file
climate_type_df = pd.read_csv("climate_type_region.csv")
climate_type_df = climate_type_df[["region_name", "climate_type"]]
# Drop the whole row if there are missing values in any column
climate_type_df = climate_type_df.dropna(subset=["region_name", "climate_type"], how='all')
# Drop the whole row if there are 'None' values in any column
climate_type_df = climate_type_df.replace('None', pd.NA).dropna()
climate_type_df.head()

,region_name,climate_type
0,Ilocos Norte,Type II
1,Ilocos Sur,Type II
2,La Union,Type I
3,Pangasinan,Type I
4,Batanes,Type I


In [7]:
# 4. Merge the shapefile GeoDataFrame with the climate type per pregion
# Assuming the 'region_name' column in rainfall_df matches a column in the shapefile (like 'NAME_1' or 'region')
merged_data = ph_shapefile.merge(
    climate_type_df, left_on="adm2_en", right_on="region_name", how="left"
)

# 5. Define the color mapping for climate types
climate_type_colors = {
    "Type I": "#A6D0E4",  # Pastel blue
    "Type II": "#B8E6B8",  # Pastel green
    "Type III": "#FFB3BA",  # Pastel red
    "Type IV": "#FFFFD1",  # Pastel yellow
}

# 6. Create a column in the GeoDataFrame to hold the color based on climate type
merged_data["color"] = merged_data["climate_type"].map(climate_type_colors)
merged_data.head()

,adm2_en,geo_level,geometry,region_name,climate_type,color
0,Ilocos Norte,Prov,"POLYGON ((285928.399 2055561.259, 285954.238 2...",Ilocos Norte,Type II,#B8E6B8
1,Ilocos Sur,Prov,"MULTIPOLYGON (((242575.411 1968180.313, 242419...",Ilocos Sur,Type II,#B8E6B8
2,La Union,Prov,"POLYGON ((240302.245 1844607.162, 239281.38 18...",La Union,Type I,#A6D0E4
3,Pangasinan,Prov,"MULTIPOLYGON (((171359.2 1820090.799, 171358.1...",Pangasinan,Type I,#A6D0E4
4,Batanes,Prov,"MULTIPOLYGON (((390824.572 2333905.124, 390822...",Batanes,Type I,#A6D0E4


In [8]:
# # Download merged_data to a CSV file
# merged_data.to_csv("merged_climate_data.csv", index=False)
# print("Data has been saved to 'merged_climate_data.csv'")

In [9]:
merged_data["region_name"].unique()
len_unique_regions = len(merged_data["region_name"].unique())
print(len_unique_regions)

88


In [10]:
merged_data

,adm2_en,geo_level,geometry,region_name,climate_type,color
0,Ilocos Norte,Prov,"POLYGON ((285928.399 2055561.259, 285954.238 2...",Ilocos Norte,Type II,#B8E6B8
1,Ilocos Sur,Prov,"MULTIPOLYGON (((242575.411 1968180.313, 242419...",Ilocos Sur,Type II,#B8E6B8
2,La Union,Prov,"POLYGON ((240302.245 1844607.162, 239281.38 18...",La Union,Type I,#A6D0E4
3,Pangasinan,Prov,"MULTIPOLYGON (((171359.2 1820090.799, 171358.1...",Pangasinan,Type I,#A6D0E4
4,Batanes,Prov,"MULTIPOLYGON (((390824.572 2333905.124, 390822...",Batanes,Type I,#A6D0E4
...,...,...,...,...,...,...
83,Sulu,Prov,"MULTIPOLYGON (((245461.226 707396.353, 245457....",Sulu,Type IV,#FFFFD1
84,Tawi-Tawi,Prov,"MULTIPOLYGON (((-6170.646 795679.841, -6173.76...",Tawi-Tawi,Type IV,#FFFFD1
85,Maguindanao del Norte,Prov,"MULTIPOLYGON (((660243.194 820832.066, 655972....",Maguindanao del Norte,Type IV,#FFFFD1
86,Maguindanao del Sur,Prov,"POLYGON ((689022.651 794303.039, 689932.162 79...",Maguindanao del Sur,Type IV,#FFFFD1


In [11]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   adm2_en       87 non-null     object  
 1   geo_level     86 non-null     object  
 2   geometry      88 non-null     geometry
 3   region_name   87 non-null     object  
 4   climate_type  87 non-null     object  
 5   color         87 non-null     object  
dtypes: geometry(1), object(5)
memory usage: 4.8+ KB


In [12]:
# Create a DataFrame showing columns with null or missing values
null_counts = pd.DataFrame(merged_data.isnull().sum(), columns=["Null Count"])
null_counts = null_counts[null_counts["Null Count"] > 0].sort_values(
    "Null Count", ascending=False
)
null_counts.reset_index(inplace=True)
null_counts.columns = ["Column", "Null Count"]

if null_counts.empty:
    print("There are no null or missing values in the DataFrame.")
else:
    print("Columns with null or missing values:")
    print(null_counts)

# Display the total number of rows with at least one null value
total_null_rows = merged_data.isnull().any(axis=1).sum()
print(f"\nTotal rows with at least one null value: {total_null_rows}")

# Display a sample of rows with null values (if any)
if total_null_rows > 0:
    print("\nSample of rows with null values:")
    print(merged_data[merged_data.isnull().any(axis=1)].head())

# Display the DataFrame
null_counts

Columns with null or missing values:
         Column  Null Count
0     geo_level           2
1       adm2_en           1
2   region_name           1
3  climate_type           1
4         color           1

Total rows with at least one null value: 2

Sample of rows with null values:
                             adm2_en geo_level  \
46  City of Isabela (Not a Province)      None   
87                              None      None   

                                             geometry  \
46  MULTIPOLYGON (((388524.379 745811.202, 388525....   
87  MULTIPOLYGON (((655158.797 814144.704, 655972....   

                         region_name climate_type    color  
46  City of Isabela (Not a Province)     Type III  #FFB3BA  
87                               NaN          NaN      NaN  


,Column,Null Count
0,geo_level,2
1,adm2_en,1
2,region_name,1
3,climate_type,1
4,color,1


In [18]:
# Isolate the map
list_of_provinces = merged_data["adm2_en"].unique().tolist()[:80]


selected_provinces = merged_data[merged_data["adm2_en"].isin(list_of_provinces)]
selected_provinces.head()


,adm2_en,geo_level,geometry,region_name,climate_type,color
0,Ilocos Norte,Prov,"POLYGON ((285928.399 2055561.259, 285954.238 2...",Ilocos Norte,Type II,#B8E6B8
1,Ilocos Sur,Prov,"MULTIPOLYGON (((242575.411 1968180.313, 242419...",Ilocos Sur,Type II,#B8E6B8
2,La Union,Prov,"POLYGON ((240302.245 1844607.162, 239281.38 18...",La Union,Type I,#A6D0E4
3,Pangasinan,Prov,"MULTIPOLYGON (((171359.2 1820090.799, 171358.1...",Pangasinan,Type I,#A6D0E4
4,Batanes,Prov,"MULTIPOLYGON (((390824.572 2333905.124, 390822...",Batanes,Type I,#A6D0E4


In [14]:
print(len(selected_provinces))

88


In [19]:
# 7. Create the base map centered on the Philippines
map_center = [12.8797, 121.7740]  # Center of the Philippines
climate_map = folium.Map(location=map_center, zoom_start=5.5, tiles="cartodbpositron")

# add MiniMap and MeasureControl (combined for slight efficiency gain)
climate_map.add_child(MiniMap())
climate_map.add_child(MeasureControl())


# Function to style the GeoJson features (no changes needed)
def style_function(feature):
    climate_type = feature["properties"]["climate_type"]
    return {
        "fillColor": climate_type_colors.get(climate_type, "gray"),
        "color": "black",
        "weight": 1.5,
        "fillOpacity": 0.7,
    }


# Convert merged_data to GeoJSON once (already efficient)
geojson_data = selected_provinces.to_crs(epsg=4326).__geo_interface__

# Create the GeoJson layer once and reuse it
geojson_layer = folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["region_name", "climate_type"]),
)

# Add GeoJson to the map
geojson_layer.add_to(climate_map)

# Fit map bounds using the pre-created layer
climate_map.fit_bounds(geojson_layer.get_bounds())

# Add a legend (no changes needed, already efficient)
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 120px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            ">&nbsp; Climate Types <br>
    &nbsp; <i class="fa fa-square fa-1x" style="color:{}"></i> Type I <br>
    &nbsp; <i class="fa fa-square fa-1x" style="color:{}"></i> Type II <br>
    &nbsp; <i class="fa fa-square fa-1x" style="color:{}"></i> Type III <br>
    &nbsp; <i class="fa fa-square fa-1x" style="color:{}"></i> Type IV
</div>
""".format(
    climate_type_colors["Type I"],
    climate_type_colors["Type II"],
    climate_type_colors["Type III"],
    climate_type_colors["Type IV"],
)

climate_map.get_root().html.add_child(folium.Element(legend_html))

# Show the map
# climate_map

In [20]:
# Save the map as an HTML file
output_file = "/Users/reneboygarcia/Downloads/ph_climate_map_80.html"
climate_map.save(output_file)

print(f"Map saved as {output_file}")

Map saved as /Users/reneboygarcia/Downloads/ph_climate_map_80.html
